# Skin Tone Detection and Analysis

This notebook implements enhanced skin tone detection using multiple color space rules (RGB, HSV, YCrCb) and K-means clustering to identify and analyze skin tones in images.

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.cluster import KMeans
from collections import Counter
import colorsys
from IPython.display import HTML, display
import matplotlib.pyplot as plt
from skimage import color

## Skin Color Detection Function

Uses multiple color space rules to determine if a pixel is likely to be skin color.

In [ ]:
def is_likely_skin_color(r, g, b):
    """
    Enhanced skin color detection using multiple color space rules
    """
    # Convert to different color spaces for better detection
    rgb = np.uint8([[[b, g, r]]])
    hsv = cv2.cvtColor(rgb, cv2.COLOR_BGR2HSV)[0][0]
    ycrcb = cv2.cvtColor(rgb, cv2.COLOR_BGR2YCrCb)[0][0]
    
    # Normalized RGB
    sum_rgb = float(r) + float(g) + float(b)
    if sum_rgb == 0:
        return False
    
    nr = r / sum_rgb
    ng = g / sum_rgb
    nb = b / sum_rgb
    
    # HSV thresholds
    h, s, v = hsv[0], hsv[1], hsv[2]
    
    # YCrCb thresholds
    y, cr, cb = ycrcb[0], ycrcb[1], ycrcb[2]
    
    # Combined rules for skin detection
    rgb_rule = (r > 95 and g > 40 and b > 20 and 
                max(r, g, b) - min(r, g, b) > 15 and
                abs(r - g) > 15 and r > g and r > b)
    
    normalized_rule = (nr > 0.35 and nb < 0.35 and 
                      nr > ng and ng > nb and 
                      abs(nr - ng) < 0.35)
    
    hsv_rule = (h >= 0 and h <= 50 and 
                s >= 10 and s <= 180 and 
                v >= 80)
    
    ycrcb_rule = (cr >= 135 and cr <= 180 and 
                  cb >= 85 and cb <= 135)
    
    return (rgb_rule and normalized_rule) or (hsv_rule and ycrcb_rule)

## Cluster Selection Function

Selects the dominant cluster with better weighting for skin-like clusters.

In [ ]:
def get_dominant_non_dark_cluster(pixels, labels, n_clusters):
    """
    Enhanced cluster selection with better weighting for skin-like clusters
    """
    clusters = []
    for i in range(n_clusters):
        cluster_pixels = pixels[labels == i]
        if len(cluster_pixels) == 0:
            continue
            
        median_color = np.median(cluster_pixels, axis=0)
        mean_color = np.mean(cluster_pixels, axis=0)
        
        # Calculate cluster metrics
        brightness = np.mean(median_color)
        color_variance = np.var(cluster_pixels, axis=0).mean()
        size = len(cluster_pixels)
        
        # Score how skin-like the cluster is
        rgb = tuple(int(x * 255) for x in median_color)
        is_skin = is_likely_skin_color(*rgb[::-1])  # Reverse RGB to BGR
        
        # Calculate cluster score
        score = (0.4 * brightness +  # Prefer brighter clusters
                0.3 * (1 - color_variance) +  # Prefer uniform clusters
                0.3 * (size / len(pixels)))   # Prefer larger clusters
                
        if is_skin:
            score *= 1.5  # Boost score for skin-like clusters
            
        clusters.append((i, score, brightness, size, median_color))
    
    # Sort by score
    clusters.sort(key=lambda x: -x[1])
    
    # Return first cluster that meets minimum criteria
    for cluster_id, score, brightness, size, median_color in clusters:
        if brightness > 0.15 and size > len(pixels) * 0.05:  # Must be bright enough and large enough
            rgb = tuple(int(x * 255) for x in median_color)
            if is_likely_skin_color(*rgb[::-1]):
                return cluster_id
    
    return clusters[0][0]  # Fallback to highest scored cluster

## Color Space Conversion and Categorization

Functions to convert RGB to L*a*b* color space and categorize lightness values.

In [ ]:
def rgb_to_lab(r, g, b):
    """Convert RGB values to L*a*b* color space and return L* value"""
    rgb = np.array([[[(r/255), (g/255), (b/255)]]])
    lab = color.rgb2lab(rgb)
    return lab[0, 0, 0]  # Return just the L* value

def get_lightness_description(L_value):
    """Get a descriptive category for the L* value"""
    if L_value >= 85:
        return "Very Light"
    elif L_value >= 70:
        return "Light"
    elif L_value >= 55:
        return "Medium Light"
    elif L_value >= 40:
        return "Medium"
    elif L_value >= 25:
        return "Medium Dark"
    elif L_value >= 10:
        return "Dark"
    else:
        return "Very Dark"

## Main Analysis Function

Enhanced skin detection with preprocessing and feature extraction.

In [ ]:
def display_skin_analysis(image_path):
    """
    Enhanced skin detection with preprocessing and better feature extraction
    """
    # Read image
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    if image is None:
        raise ValueError(f"Failed to read image: {image_path}")
    
    # Split alpha channel and convert to BGR for filtering
    if image.shape[2] == 4:  # BGRA
        bgr = image[:, :, :3]
        alpha = image[:, :, 3]
        filtered = cv2.bilateralFilter(bgr, 9, 75, 75)
        # Recombine with alpha
        filtered = np.dstack((filtered, alpha))
    else:  # BGR
        filtered = cv2.bilateralFilter(image, 9, 75, 75)
    
    # Rest of the function remains the same...
    (height, width, _) = filtered.shape
    pixels = []
    valid_pixel_coords = []
    features = []
    
    for row in range(height):
        for col in range(width):
            pixel = filtered[row, col]
            if len(pixel) == 4:  # BGRA
                b, g, r, a = pixel
            else:  # BGR
                b, g, r = pixel
                a = 255
                
            if a > 127:  # Only process non-transparent pixels
                # Normalize RGB
                r_norm, g_norm, b_norm = r/255, g/255, b/255
                
                # Add HSV features
                hsv = cv2.cvtColor(np.uint8([[[b, g, r]]]), cv2.COLOR_BGR2HSV)[0][0]
                h_norm, s_norm, v_norm = hsv[0]/179, hsv[1]/255, hsv[2]/255
                
                # Add YCrCb features
                ycrcb = cv2.cvtColor(np.uint8([[[b, g, r]]]), cv2.COLOR_BGR2YCrCb)[0][0]
                y_norm, cr_norm, cb_norm = ycrcb[0]/255, ycrcb[1]/255, ycrcb[2]/255
                
                feature_vector = [
                    r_norm, g_norm, b_norm,
                    h_norm, s_norm, v_norm,
                    y_norm, cr_norm, cb_norm
                ]
                
                features.append(feature_vector)
                pixels.append([r_norm, g_norm, b_norm])
                valid_pixel_coords.append((row, col))
    
    # Rest of the function implementation remains the same...
    if not pixels:
        raise ValueError("No non-transparent pixels found in image")
    
    # Convert to numpy arrays
    pixels = np.array(pixels)
    features = np.array(features)
    
    # Perform k-means clustering with more clusters
    kmeans = KMeans(n_clusters=5, n_init=10)
    kmeans.fit(features)
    
    # Get center patch to identify skin cluster
    center_x, center_y = width // 2, height // 2
    patch_size = min(width, height) // 4  # Smaller patch size
    center_pixels = []
    
    for idx, (row, col) in enumerate(valid_pixel_coords):
        if (abs(col - center_x) < patch_size and 
            abs(row - center_y) < patch_size):
            center_pixels.append(idx)
    
    if not center_pixels:
        raise ValueError("No valid pixels found in center patch")
    
    # Get dominant non-dark cluster from center region
    center_pixels_colors = pixels[center_pixels]
    center_pixels_labels = kmeans.labels_[center_pixels]
    skin_cluster = get_dominant_non_dark_cluster(
        center_pixels_colors, 
        center_pixels_labels, 
        5
    )
    
    # Create skin detection output image
    output = np.zeros_like(image)
    for idx, (row, col) in enumerate(valid_pixel_coords):
        if kmeans.labels_[idx] == skin_cluster:
            output[row, col] = image[row, col]
        else:
            output[row, col] = [255, 240, 180, 255]
    
    # Get skin tone color from the identified skin pixels
    skin_mask = kmeans.labels_ == skin_cluster
    skin_pixels = pixels[skin_mask]
    
    if len(skin_pixels) == 0:
        raise ValueError("No skin pixels detected")
    
    # Use 95th percentile instead of median to avoid darker pixels
    bright_threshold = np.percentile(skin_pixels, 95, axis=0)
    bright_mask = np.mean(skin_pixels, axis=1) > np.mean(bright_threshold) * 0.7
    skin_pixels_filtered = skin_pixels[bright_mask]
    
    if len(skin_pixels_filtered) == 0:
        skin_pixels_filtered = skin_pixels
    
    # Calculate median color
    median_color = np.median(skin_pixels_filtered, axis=0)
    rgb = tuple(int(x * 255) for x in median_color)
    
    # Convert to L*a*b* and extract L* value
    L = rgb_to_lab(*rgb)
    lightness_category = get_lightness_description(L)
    
    # Create visualization
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    
    # Original image
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGRA2RGBA)
    axs[0].imshow(image_rgb)
    axs[0].set_title('Original Image')
    axs[0].axis('off')
    
    # Skin detection
    output_rgb = cv2.cvtColor(output, cv2.COLOR_BGRA2RGBA)
    axs[1].imshow(output_rgb)
    axs[1].set_title('Skin Detection')
    axs[1].axis('off')
    
    # Color patch
    color_patch = np.full((100, 100, 3), [rgb[0], rgb[1], rgb[2]], dtype=np.uint8)
    axs[2].imshow(color_patch)
    axs[2].set_title(f'RGB Color\n#{rgb[0]:02x}{rgb[1]:02x}{rgb[2]:02x}')
    axs[2].axis('off')
    
    # Lightness visualization
    gradient = np.linspace(0, 100, 100).reshape(1, -1)
    gradient = np.repeat(gradient, 50, axis=0)
    axs[3].imshow(gradient, cmap='Greys', aspect='auto')
    axs[3].set_title(f'Lightness (L*): {L:.1f}\n{lightness_category}')
    axs[3].axis('off')
    marker_x = int((L / 100) * 99)
    axs[3].plot(marker_x, 25, 'r^', markersize=10)
    
    plt.tight_layout()
    plt.show()
    
    print(f"Lightness (L*): {L:.1f}")
    print(f"Category: {lightness_category}")
    print(f"RGB values: rgb{rgb}")
    
    return L, lightness_category, rgb

## Batch Processing Function

Analyze multiple images in a directory and save results to CSV.

In [ ]:
def analyze_directory(directory_path, output_csv=None):
    """
    Analyze and display results for all PNG files in a directory and save results to CSV
    """
    results = []
    
    if output_csv is None:
        output_csv = os.path.join(directory_path, 'skin_tones.csv')
    
    for filename in sorted(os.listdir(directory_path)):
        if filename.lower().endswith('.png'):
            print(f"\nAnalyzing {filename}:")
            image_path = os.path.join(directory_path, filename)
            
            try:
                L, category, rgb = display_skin_analysis(image_path)
                
                results.append({
                    'filename': filename,
                    'L_value': L,
                    'category': category,
                    'r': rgb[0],
                    'g': rgb[1],
                    'b': rgb[2],
                    'rgb_hex': f"#{rgb[0]:02x}{rgb[1]:02x}{rgb[2]:02x}"
                })
                
            except Exception as e:
                print(f"Error processing {filename}: {str(e)}")
                results.append({
                    'filename': filename,
                    'L_value': None,
                    'category': 'ERROR',
                    'r': None,
                    'g': None,
                    'b': None,
                    'rgb_hex': None
                })
    
    # Create and save DataFrame
    df = pd.DataFrame(results)
    df = df.sort_values('L_value', ascending=True, na_position='last')
    df.to_csv(output_csv, index=False)
    print(f"\nResults saved to: {output_csv}")
    
    return df

## Example Usage

Uncomment and run the cells below to analyze images.

In [ ]:
# For a single image:
# L, category, rgb = display_skin_analysis("path/to/face.png")

In [ ]:
# For a directory of images:
df = analyze_directory(
    r"faces",
    r"output_skintones.csv"
)